In [11]:
# Grad cam code

from __future__ import absolute_import, division, print_function
from tqdm import tqdm    

from termcolor import colored
import tensorflow as tf
import os                   # work with directories
import numpy as np          # dealing with arrays
import matplotlib.pyplot as plt
import time

tfe = tf.contrib.eager
import pickle as pk
import sys

from tensorflow.python.client import device_lib
from tensorflow.python.eager import tape
import  cv2
import platform

from skimage.transform import resize
import warnings

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
layers = tf.keras.layers
tf.enable_eager_execution(config=config)
tf.executing_eagerly()
print(tf.__version__)

machine_type = platform.uname()[0]
if machine_type == 'Linux':
    path_sep = '/'
else:
    path_sep = '\win'
HOME_DIR = os.getcwd()
MAIN_dir = HOME_DIR + path_sep+'Cleaned_code'+path_sep+'Shared_git'+path_sep
checkpoint_path = MAIN_dir + path_sep+'utils'+path_sep+'checkpoints'+path_sep
data_files_path = MAIN_dir + 'data_files'+path_sep
mean_path = data_files_path + 'std_mean_60_dataset.npy'
dataset_path = MAIN_dir + 'dataset'+path_sep
output_path = MAIN_dir+'utils'+ path_sep+'output'+ path_sep+'gracam.png'


1.12.0


In [12]:

filename = 'sample_set.pickle'
pickle_in_sample = open(MAIN_dir + 'data_files/' + filename, "rb")
sample_data = pk.load(pickle_in_sample)
pickle_in_sample.close()

images_s = sample_data['images']
labels_s = sample_data['labels']
speeds_s = sample_data['speeds']
log_length_s = sample_data['log_length']


In [13]:
Batch_size = 1
Buffer_size = 1
Epochs = 100

sample_dataset = tf.data.Dataset.from_tensor_slices((images_s, labels_s, speeds_s, log_length_s))
sample_dataset = sample_dataset.batch(Batch_size)
sample_dataset = sample_dataset.prefetch(Batch_size)


In [14]:
# LOAD RES-9ER - default
from Cleaned_code.Shared_git.utils.models import RES_9ER_CAM as Res

# Instantiate the model and configure tensorbaord and checkpoints
data_format = 'channels_last'
model = Res.Res9ER(data_format=data_format, include_top=True, pooling=None, classes=1)
optimizer = tf.train.AdamOptimizer()
accuracy = 0 
acc_prediction = tf.constant(0, dtype="float32")
t = tf.constant(1/7.5, dtype="float32")
loss = tf.constant(0, dtype="float32")



In [15]:
# LOAD checkpoint if you wish to test results
checkpoint_name = 'Gradcam_RES_9ER'
model.load_weights(checkpoint_path+checkpoint_name)


In [16]:
def compute_loss(prediction, label, train_log_length_, operation='L2'):
    if operation == 'L2':
        return tf.divide(tf.squared_difference(prediction, label), train_log_length_)
    elif operation == 'Subtraction':
        return tf.divide(tf.subtract(prediction, label), train_log_length_)
    elif operation == 'L1':
        return tf.divide(tf.abs(tf.subtract(prediction, label)), train_log_length_)
    else:
        raise ValueError('Please specify loss function (L2, L1, Subtraction)')


def print_progress(count, total, cnt, overall, time_, count_log, loss, loss_):
    percent_complete = float(count) / total
    overall_complete = float(cnt) / (overall-1)

    sec = time_ % 60
    mint = int(time_/60) % 60
    hr = int(time_/3600) % 60
    loss = str(loss)
    loss_ = str(loss_)
    msg = "\r Time_lapsed (hr:mm:ss) --> {0:02d}:{1:02d}:{2:02d} ,   loss: {3:s}   Log Progress: {4:.1%},     Overall Progress:{5:.1%}," \
        " completed {6:d} out of 185 logs <--> Initial loss: {7:s} ".format(hr, mint, sec, loss, percent_complete, overall_complete, count_log, loss_)
    sys.stdout.write(msg)
    sys.stdout.flush()
    

def validation_progress(log_cnt, num_logs, time_, loss, accuracy_loc):
    log_cnt += 1
    overall_complete = float(log_cnt) / num_logs
    sec = int(time_) % 60
    mint = int(time_/60) % 60
    hr = int(time_/3600) % 60
    loss = str(loss)
    msg = "\r Validation_Time (hr:mm:ss) --> {0:02d}:{1:02d}:{2:02d} ,   Avg_loss: {3:s}   Avg_accuracy: {4:.2%}   Overall Progress:{5:.1%}," \
        " completed {6:d} out of {7:d} logs".format(hr, mint, sec, loss, accuracy_loc, overall_complete, log_cnt, num_logs)
    sys.stdout.write(colored(msg, 'green'))
    sys.stdout.flush()


def write_summaries(loss, i, global_step, vars_loc, grads_loc, train=True):
    with summary_writer.as_default():
        with tf.contrib.summary.always_record_summaries():
            if train:
                tf.contrib.summary.scalar("train_loss", loss, step=global_step)
                tf.contrib.summary.scalar("step", i, step=global_step)
                #  do not add spaces after names
                tf.contrib.summary.histogram("weights", vars_loc, step=global_step)
                tf.contrib.summary.histogram("gradients", grads_loc, step=global_step)
            else:
                tf.contrib.summary.scalar("val_loss", loss, step=global_step)
                




In [17]:

def visualize_mod(img, cam, filename):
  
    fig, ax = plt.subplots(nrows=1, ncols=3)

    plt.subplot(121)
    plt.axis("off")
    imgplot = plt.imshow(img)

    cam = (cam*-1.0) + 1.0
    cam_heatmap = np.array(cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET))
    # plt.subplot(143)
    plt.axis("off")

    plt.subplot(122)
    plt.axis("off")
    
    cam_heatmap = cam_heatmap/255.0

    fin = (img*0.7) + (cam_heatmap*0.3)
    imgplot = plt.imshow(fin)

    plt.savefig(filename, dpi=600)
    plt.close(fig)


In [35]:
# TRAINING CODE

warnings.filterwarnings('ignore')
start = time.time()
performance_flag = True
img_idx = 40  # image to visualize
prediction_raw = []

for (batch, (image, label, speed, lengths)) in (enumerate(sample_dataset)):
    size = np.shape(label)[0]
    labels = tf.reshape(label, [size, 1])
    speeds = tf.reshape(speed, [size, 1])
    log_length = tf.squeeze(lengths)
    # images = tf.squeeze(image)
    # images = tf.expand_dims(images)
    images = tf.reshape(image, [1,96,144,3])
    batch += 1
    if batch >= img_idx and batch <= (img_idx+288):
        step = tf.train.get_or_create_global_step()
        cost, layer_conv = model(images, conv_out= True)

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(images)
            cost, layer_conv = model(images, conv_out= True)
            prediction_raw.append(cost)

        gb_grad = tape.gradient(cost, images)
        target_conv_layer_grad = tape.gradient(cost, layer_conv)
        del tape
        conv_first_grad = tf.exp(cost)*target_conv_layer_grad 	
        #second_derivative
        conv_second_grad = tf.exp(cost)*target_conv_layer_grad*target_conv_layer_grad 
    
        #triple_derivative
        conv_third_grad = tf.exp(cost)[0]*target_conv_layer_grad*target_conv_layer_grad*target_conv_layer_grad
        global_sum = np.sum(tf.reshape(layer_conv[0],(-1,conv_first_grad[0].shape[2])), axis=0)
       
        alpha_num = conv_second_grad[0]
        alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
        alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
        alphas = alpha_num/alpha_denom

        weights = np.maximum(conv_first_grad[0], 0.0)
        
        alphas_thresholding = np.where(weights, alphas, 0.0)
        
        alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
        alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))
        alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[0].shape[2]))
    
    
        
        deep_linearization_weights = np.sum(tf.reshape((weights*alphas),(-1,conv_first_grad[0].shape[2])),axis=0)
        
        #print deep_linearizat0=eglmoprstvion_weights
        grad_CAM_map = np.sum(deep_linearization_weights*layer_conv[0], axis=2)
        
        # Passing through ReLU
        cam = np.maximum(grad_CAM_map, 0)
        cam = cam / np.max(cam) # scale 0 to 1.0   
    
        cam = resize(cam, (480, 720))
        # Passing through ReLU
        cam = np.maximum(grad_CAM_map, 0)
        
        cam = cam / np.max(cam) # scale 0 to 1.0    
        cam = resize(cam, (480, 720))
        img = plt.imread(images_s[(img_idx-1)*Batch_size])
        img2 = resize(img, (480, 720))
        plt.imshow(img2)
        visualize_mod(img2, cam, output_path)
    if batch > img_idx:
        break


TypeError: Object does not appear to be a 8-bit string path or a Python file-like object